In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/moons"
results_dir_relu = "results/classification/single_layer/relu/moons"
results_dir_tanh = "results/classification/single_layer/tanh/moons"
model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T", "Pred CP"]
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh" , "Pred CP tanh"]


relu_fits = {}
tanh_fits = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu,
        include_prior=False,
    )
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    

    relu_fits[base_config_name] = relu_fit  # use clean key
    tanh_fits[base_config_name] = tanh_fit  # use clean key
    

In [ ]:
data_dir = f"datasets/moons/many"

results_dir_relu = "results/classification/single_layer/relu/moons"
results_dir_tanh = "results/classification/single_layer/tanh/moons"
model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]

base_config_name = "Moon_N500_p2_sigma0.20_seed11"
full_config_path = "Moon_N500_p2_sigma0.20_seed11"
relu_fit = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_relu,
    models=model_names_relu,
    include_prior=False,
)

tanh_fit = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_tanh,
    models=model_names_tanh,
    include_prior=False,
)
    

relu_fits[base_config_name] = relu_fit  # use clean key
tanh_fits[base_config_name] = tanh_fit  # use clean key
    

In [4]:
import re
from utils.generate_data import generate_two_moons
from utils.robust_utils import build_pytorch_model_from_stan_sample
def evaluate_on_generated_test_set(
    fits_dict,
    input_dim,
    hidden_dim,
    output_dim,
    N_test=1000,
    sigma=0.2,
    D=2,
    seed=999,
    sample_indices=range(0, 2000, 50),
):
    import torch
    import torch.nn.functional as F
    from sklearn.metrics import accuracy_score, log_loss

    _, X_test, _, y_test = generate_two_moons(N=N_test, sigma=sigma, test_size=0.99, D=D, seed=seed)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test = np.asarray(y_test)

    results = {}

    for model_name, model_entry in fits_dict.items():
        fit = model_entry["posterior"]

        probs = []

        for idx in sample_indices:
            model = build_pytorch_model_from_stan_sample(
                fit=fit,
                sample_idx=idx,
                input_dim=input_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                task="classification"
            )
            model.eval()
            with torch.no_grad():
                logits = model(X_test_tensor)
                prob = F.softmax(logits, dim=1).numpy()
                probs.append(prob)

        probs = np.stack(probs, axis=0)  # shape: (n_samples, N_test, 2)
        mean_probs = np.mean(probs, axis=0)
        y_pred_mode = np.argmax(mean_probs, axis=1) + 1

        acc = accuracy_score(y_test, y_pred_mode)
        nll = log_loss(y_test - 1, mean_probs)

        results[model_name] = {
            "accuracy": acc,
            "nll": nll
        }
    return results, X_test, y_test

def extract_metadata_from_key(key):
    """Hent ut N og seed fra nøkkel som 'Moon_N100_p2_sigma0.20_seed1'"""
    match = re.search(r"N(\d+)_p\d+_sigma[\d\.]+_seed(\d+)", key)
    if match:
        return int(match.group(1)), int(match.group(2))
    else:
        return None, None

def evaluate_all_models_on_generated_test_set(fits_dict, N_test=200, sample_indices=range(2000)):
    all_results = []

    for key, fits in fits_dict.items():
        N, seed = extract_metadata_from_key(key)
        if N is None:
            continue

        result, _, _ = evaluate_on_generated_test_set(
            fits_dict=fits,
            input_dim=2,
            hidden_dim=16,
            output_dim=2,
            N_test=N_test,
            sigma=0.2,
            D=2,
            seed=999,  # fast seed for fair test-set
            sample_indices=sample_indices
        )

        for model_name, metrics in result.items():
            all_results.append({
                "config": key,
                "model": model_name,
                "N": N,
                "seed": seed,
                "accuracy": metrics["accuracy"],
                "nll": metrics["nll"]
            })

    df = pd.DataFrame(all_results)
    return df


In [5]:
df_results = evaluate_all_models_on_generated_test_set(relu_fits, N_test=500)


In [ ]:
summary = df_results.groupby(["model", "N"]).agg(
    acc_mean=("accuracy", "mean"),
    #acc_std=("accuracy", "std"),
    nll_mean=("nll", "mean"),
    #nll_std=("nll", "std"),
).reset_index()

print(summary)


In [7]:
_, X_test, _, y_test = generate_two_moons(N=500, sigma=0.2, test_size=0.99, D=2, seed=999)

# Sample 25% of the test set
X_test_df = pd.DataFrame(X_test)
y_test_s = pd.Series(y_test) - 1

test_subset = X_test_df.sample(frac=1.0, random_state=42)
subset_indices = test_subset.index
y_subset = y_test_s.loc[subset_indices] 

#sample_indices = range(0, 2000, 50)  # or whatever stride you like
sample_indices = range(0, 4000, 800)  
input_dim = X_test.shape[1]
hidden_dim = 16
output_dim = 2
epsilon = 0.1
delta = 0.1
p_norm = 2

from utils.robust_utils import estimate_robustness_over_test_set

epsilons = [0.01, 0.05, 0.1, 0.25]
scales = [0.01, 0.05, 0.1, 0.5, 1.0, 5.0]


In [ ]:
from collections import defaultdict
import pandas as pd

# Collect results
all_results = []

# These are your training config keys (not model names)
model_keys = [
    "Moon_N100_p2_sigma0.20_seed1",
    "Moon_N200_p2_sigma0.20_seed2",
    "Moon_N500_p2_sigma0.20_seed11"
]

# Loop over each training configuration
for key in model_keys:
    N, seed = extract_metadata_from_key(key)
    fits_dict = relu_fits[key]  # <-- dict with model names inside

    for model_name, fit_entry in fits_dict.items():
        # You now iterate over actual models like "Gaussian", "Dirichlet Horseshoe", etc.

        for epsilon in epsilons:
            for scale in scales:
                delta = scale * epsilon

                df_result = estimate_robustness_over_test_set(
                    x_test=test_subset,
                    y_test=y_subset,
                    fits_dict={model_name: fit_entry},  # Wrap in dict to reuse code
                    input_dim=input_dim,
                    hidden_dim=hidden_dim,
                    output_dim=output_dim,
                    sample_indices=sample_indices,
                    epsilon=epsilon,
                    delta=delta,
                    p_norm=p_norm,
                )

                df_result["epsilon"] = epsilon
                df_result["delta"] = delta
                df_result["N"] = N
                df_result["seed"] = seed
                df_result["model"] = model_name  # ✅ Store correct model name
                all_results.append(df_result.copy())

# Combine all into DataFrame
df_robust = pd.concat(all_results, ignore_index=True)

# Unpack robustness dictionary into columns
df_flat = pd.concat(
    [df_robust.drop(columns=["robustness"]),
     df_robust["robustness"].apply(pd.Series)],
    axis=1
)

# Add derived columns
df_flat["1-p1"] = (1.0 - df_flat["p1"]).round(5)
df_flat["1-p2"] = (1.0 - df_flat["p2"]).round(5)
df_flat["scale"] = (df_flat["delta"] / df_flat["epsilon"]).round(2)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Ensure model name consistency and ordering
model_order = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
Ns = sorted(df_flat["N"].unique())

# Global color scale (consistent across all plots)
vmin = df_flat["1-p1"].min()
vmax = df_flat["1-p1"].max()

for N in Ns:
    df_N = df_flat[df_flat["N"] == N]

    # Create subplots: 2 rows x 2 cols
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8), sharex=True, sharey=True)

    for j, model in enumerate(model_order):
        row, col = divmod(j, 2)
        ax = axes[row, col]
        df_model = df_N[df_N["model"] == model]

        # Use pivot_table to handle duplicates
        heatmap_df = df_model.pivot_table(
            index="scale", columns="epsilon", values="1-p1", aggfunc="mean"
        )

        sns.heatmap(
            heatmap_df.sort_index(ascending=False),
            annot=True, fmt=".2f", cmap="RdYlGn", ax=ax,
            cbar=False, vmin=vmin, vmax=vmax
        )

        ax.set_title(model, fontsize=13)
        ax.set_xlabel(r"$\varepsilon$", fontsize=12)
        ax.set_ylabel(r"$\delta / \varepsilon$", fontsize=12)

    # Adjust layout to leave space for colorbar
    plt.tight_layout(rect=[0, 0, 0.93, 0.95])

    # Add colorbar on the right
    cbar_ax = fig.add_axes([0.94, 0.25, 0.015, 0.5])  # [left, bottom, width, height]
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    sm = plt.cm.ScalarMappable(cmap="RdYlGn", norm=norm)
    sm.set_array([])
    fig.colorbar(sm, cax=cbar_ax, label='$1 - p_1$')

    fig.suptitle(f"Softmax Shift Robustness (1 - $p_1$), N = {N}", fontsize=16)
    plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure correct model order for consistency
model_order = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
palette = sns.color_palette("Set2", n_colors=4)

# Optional: choose one N (or loop if needed)
N_select = 100
df_plot = df_flat[df_flat["N"] == N_select]

plt.figure(figsize=(10, 6))
sns.violinplot(
    data=df_plot,
    x="epsilon",
    y="1-p2",
    hue="model",
    hue_order=model_order,
    palette=palette,
    cut=0,
    inner="quartile",
    linewidth=1
)

plt.title(f"Label Invariance Probability ($1 - p_2$) across models, N = {N_select}")
plt.xlabel(r"Adversarial strength $\varepsilon$")
plt.ylabel(r"$1-p_2$")
plt.ylim(0, 1)
plt.legend(title="Model", bbox_to_anchor=(0.02, 0.05))
plt.tight_layout()
plt.grid(True, axis="y")
plt.show()



In [ ]:
print("W1 shape:", relu_fits['Moon_N100_p2_sigma1.00_seed1']['Dirichlet Horseshoe']['posterior'].stan_variable("W_1").shape)
print("b1 shape:", relu_fits['Moon_N100_p2_sigma1.00_seed1']['Dirichlet Horseshoe']['posterior'].stan_variable("hidden_bias").shape)
print("W2 shape:", relu_fits['Moon_N100_p2_sigma1.00_seed1']['Dirichlet Horseshoe']['posterior'].stan_variable("W_L").shape)
print("b2 shape:", relu_fits['Moon_N100_p2_sigma1.00_seed1']['Dirichlet Horseshoe']['posterior'].stan_variable("output_bias").shape)

In [15]:
def plot_decision_boundary_from_stan_full_posterior(
    fit,
    X_train,
    y_train,
    input_dim=2,
    hidden_dim=20,
    output_dim=2,
    task="classification",
    resolution=300,
    title="BNN decision boundary (posterior mean)",
    ax=None  # <--- new
):
    import torch
    import matplotlib.pyplot as plt
    import numpy as np
    from matplotlib.colors import ListedColormap
    import torch.nn.functional as F

    y_train = np.asarray(y_train)
    if y_train.min() == 1:
        y_train -= 1  # convert to 0-1

    x_min, x_max = X_train[:, 0].min() - 0.5, X_train[:, 0].max() + 0.5
    y_min, y_max = X_train[:, 1].min() - 0.5, X_train[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, resolution),
                         np.linspace(y_min, y_max, resolution))
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_tensor = torch.tensor(grid, dtype=torch.float32)

    n_samples = fit.stan_variable("W_1").shape[0]
    preds = []
    for idx in range(n_samples):
        model = build_pytorch_model_from_stan_sample(
            fit=fit,
            sample_idx=idx,
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            task=task
        )
        model.eval()
        with torch.no_grad():
            logits = model(grid_tensor)
            probs = F.softmax(logits, dim=1)[:, 1]
            preds.append(probs.numpy())

    mean_probs = np.mean(preds, axis=0)
    Z = mean_probs.reshape(xx.shape)

    # Create new figure/axis if not provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 5))

    cmap_pts = ListedColormap(['#FF0000', '#0000FF'])

    contour = ax.contourf(xx, yy, Z, levels=np.linspace(0, 1, 25), cmap="RdBu", alpha=0.8)
    ax.contour(xx, yy, Z, levels=[0.5], colors="black", linewidths=1.5)
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cmap_pts, edgecolor='k', s=30)
    ax.set_title(title)
    ax.set_xlabel("$x_1$")
    ax.set_ylabel("$x_2$")
    return contour  # so you can attach colorbars later


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
titles = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
fits = [relu_fits["Moon_N100_p2_sigma0.20_seed1"][model]["posterior"] for model in titles]

for ax, title, fit in zip(axs.flat, titles, fits):
    plot_decision_boundary_from_stan_full_posterior(
        fit=fit,
        X_train=X_test,
        y_train=y_test,
        input_dim=2,
        hidden_dim=16,
        output_dim=2,
        title=title,
        ax=ax
    )

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
titles = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
fits = [relu_fits["Moon_N200_p2_sigma0.20_seed2"][model]["posterior"] for model in titles]

for ax, title, fit in zip(axs.flat, titles, fits):
    plot_decision_boundary_from_stan_full_posterior(
        fit=fit,
        X_train=X_test,
        y_train=y_test,
        input_dim=2,
        hidden_dim=16,
        output_dim=2,
        title=title,
        ax=ax
    )

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
titles = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
fits = [relu_fits["Moon_N500_p2_sigma0.20_seed11"][model]["posterior"] for model in titles]

for ax, title, fit in zip(axs.flat, titles, fits):
    plot_decision_boundary_from_stan_full_posterior(
        fit=fit,
        X_train=X_test,
        y_train=y_test,
        input_dim=2,
        hidden_dim=16,
        output_dim=2,
        title=title,
        ax=ax
    )

plt.tight_layout()
plt.show()
